# Approche : 

Nous allons utiliser une approche basée sur la distance de Leveishtein, une distance mathématique donnant une mesure de la similarité entre deux chaînes de caractères. Elle est égale au nombre minimal de caractères qu'il faut supprimer, insérer ou remplacer pour passer d’une chaîne à l’autre.

Nous comparons chacune des alertes avec l'ensemble des titres de produits et récupérons le produit avec le plus petit score. 

En parcourant l'ensemble des alertes et en effectuant quelques tests, plusieurs choses nous sautent aux yeux : 
- Plusieurs fois, les clients omettent la marque du produit en mentionnant seulement le modèle, exemple : Galaxy S3 au lieu de Samsung Galaxy S3. 
- Les clients ont tendance à ne pas aller dans le détail du produit en question, exemple : Iphone 7 au lieu de "IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ "
- Les clients ont tendence à coller marque et modèle, exemple : Iphone7
- La distance de Leveinshtein met quelques secondes à s'exécuter, jusqu'à quelques minutes si on parcoure tous nos produits.

Pour pallier à ces difficultés, nous avons quelques approches : 
- Nous remarquons que le premier mot des alertes est souvent le plus important, celui qui donne une indication sur la marque et parfois même le produit lui même. Ainsi, nous mettrons une condition (voir code, commentaire (1)) sur la présence de ce mot dans le produit. Si le mot est présent nous calculons la distance, sinon nous donnons un score élevé au produit.
- Nous ne comparerons que des chaines de même taille, nous limiterons la taille de la chaîne des produits à celle de l'article. Exemple : au lieu de comparer "Iphone 7" et  "IPHONE 7 128 GO  NOIR DE JAIS  DÉBLOQUÉ ", nous comparons "Iphone 7" et "IPHONE 7". (commentaire (2))
- Nous calculons la distance entre le produit et l'alerte qu'à partir de l'apparition du premier mot dans le produit. Exemple : Au lieu de comparer  "Galaxy S3" et "Samsung Galaxy S3 Noir 16GB" nous comparons "Galaxy S3" et "Galaxy S3 Noir 16GB". (commentaire (3))
- Nous séparons le premier chiffre (Souvent le modèle) du reste de la chaine : iphone7 > iphone 7

Nous modifierons aussi les textes en mettant le tout en majuscule et en retirant quelques caractères spéciaux

# Imports

In [44]:
import pandas as pd 
import numpy as np 
import distance
import re

# Observation de nos données 

On regarde le fichier français : 

In [69]:
df_products = pd.read_csv("HOME_MISSION_SHARE/Recomendations/products_data_fr_fr_2017-11-14_150627.187511.csv", \
                          delimiter=";", encoding="latin1")
df_alerts = pd.read_csv("HOME_MISSION_SHARE/Recomendations/alerts_fr_fr_2017-11-14_151447.572712.csv", \
                        delimiter=";", encoding="latin1")

Les produits : 

In [70]:
df_products.head()

,title_fr_fr,id,description_fr_fr
0,HTC Windows 8S 4 Go - Noir - Bouygues,222,"\tSmartphone sous Android 2.1, le WildFire vie..."
1,Nokia Lumia 1020 32 Go - Blanc - SFR,228,Compatible 4G - Windows Phone 8 - Ecran AMOLED...
2,iPhone 4 16 Go - Noir - Débloqué,1178,<p><b>iPhone 4 16 Go - Noir Débloqué</b></p>\...
3,iPhone 4 16 Go - Blanc - Débloqué,1179,<strong> Le mobile sera livré sous le dernier ...
4,iPhone 4 32 Go - Noir - Débloqué,1180,<p><b>iPhone 4 32 Go - Noir - Débloqué</b></p>...


Les alertes : 

In [79]:
df_alerts

,pk,product,criterias,country_code,creation_date
0,30254,IPAD MINI 4 32 GO WIFI + 4G OR DÉBLOQUÉ,NaN,fr_fr,2017-11-14 13:09:03+00:00
1,30252,EXPRESSO À CAPSULES DOLCE GUSTO KRUPS YY1786FD,NaN,fr_fr,2017-11-14 12:18:30+00:00
2,30244,ACER SWIFT SF3145139ZJ 14 CORE I36006U 2 GHZ ...,NaN,fr_fr,2017-11-14 11:43:47+00:00
3,30226,IPAD 4 128 GO WIFI NOIR,NaN,fr_fr,2017-11-14 07:52:01+00:00
4,30217,COMPTEUR DE FANS FACEBOOK 5 CHIFFRES,NaN,fr_fr,2017-11-14 06:27:33+00:00
5,30216,ASUS ZENFONE 2 SELFIE 32 GO BLEU DÉBLOQUÉ,NaN,fr_fr,2017-11-13 23:34:03+00:00
6,30207,DORO 8031 8 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 22:02:28+00:00
7,30199,VENTILATEUR DYSON COOL AM07,NaN,fr_fr,2017-11-13 20:39:24+00:00
8,30198,LG G6 32 GO NOIR DÉBLOQUÉ,NaN,fr_fr,2017-11-13 20:42:47+00:00
9,30197,"IPAD MINI 4 7,9'' 32 GO WIFI ARGENT",NaN,fr_fr,2017-11-13 20:32:07+00:00


Quelques produits peuvent poser problème : 
- "iphone7" : il manque l'espace
- "xperia", "galaxy" : il manque le nom de marque 
- Concaténer les informations dans critères

Quelques remplacements des caractères spéciaux :

In [181]:
def words(x, word):
    try : 
        lim = word.find(re.search(r'\d', word).group(0))
        word = word[:lim]+ " "+word[lim:]
    except:
        pass
    
    #Split à la virgule pour les listes de produits : on ne prend que le premier
    if ", " in word: 
        word = word.split(',')[0]
    
    #On retire les espaces multiples
    x = re.sub('\s+', ' ', x).strip()
    word = re.sub('\s+', ' ', word).strip()
    
    #Commentaire (1) : Le premier mot est toujours le plus important dans les alertes
    if word.split()[0] in x:
        
        #Commentaire (3) : Nous chercons le mot de l'alerte pour commencer la chaine
        x = x[x.find(word.split()[0]):]
        
        #Commentaire (2) : Nous calculons la distance pour des chaines de même taille        
        return distance.levenshtein(x[:len(word)], word)
    else :
        return 999

In [196]:
#On itère sur tous nos fichiers :
for i in range(0, int(len(files)/2)):
    #import des tables : 
    df_alerts = pd.read_csv("HOME_MISSION_SHARE/Recomendations/"+files[i], delimiter=";", encoding="latin1")
    df_products = pd.read_csv("HOME_MISSION_SHARE/Recomendations/"+files[i+int(len(files)/2)], delimiter=";", encoding="latin1")
    
    #Quelques modifications sur les chaines de caractères
    df_alerts["product"] = df_alerts["product"].map(lambda x : x.upper().replace('-','')) 
    
    #On cherche le titre du produit (il n'a pas toujours la même place en fonction des langues)
    for col in df_products.columns:
        if "title" in col:
            title = col 
    
    df_products[title] = df_products[title].map(lambda x : x.upper().replace('-','')) 
    
    #On crée une nouvelle colonne :
    df_alerts["prod_correspondance"] = [[] for i in range(df_alerts.shape[0])]
    
    for j, prod in enumerate(df_alerts["product"]):
        df_products["leveinshtein"] = df_products[title].map(lambda x : words(x, prod))
        
        for item in (df_products[df_products["leveinshtein"] == df_products["leveinshtein"].min()] [title].iloc[:7].values):
            df_alerts.iloc[j, -1].append(item)
        
    df_alerts.to_csv("HOME_MISSION_SHARE/Recomendations/alerts_"+title.split("_")[-1]+".csv")
    print(df_alerts[["product", "prod_correspondance"]])

                                     product  \
0                               SAMSUNG TAB3   
1                    SAMSUNG GALAXY S7 EDGE    
2                            CANON EOS 1300D   
3                             SAMSUNG J5 (6)   
4                     SAMSUNG GALAXY 10 ZOLL   
5                            SAMSUNG J5 2015   
6                      MICROSOFT SURFACE PRO   
7               IPAD PRO 10,5 512GB WLAN/LTE   
8   GALAXY NOTE EDGE 32GB WEISS OHNE VERTRAG   
9                               NINTENDO 3DS   
10                                   MACBOOK   
11                                  MAC BOOK   
12                              NINTENDO 3DS   
13      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
14      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
15      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
16      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
17      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
18      SAMSUNG GALAXY J5, SAMSUNG GALAXY A5   
19      SAMSUNG GALAXY J5, SAMSUNG GALAX